In [1]:
from keras.models import Model, load_model
from tensorflow.keras.utils import img_to_array, load_img, array_to_img
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
import os
import numpy as np
import cv2
from tqdm import tqdm
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.metrics import MeanIoU
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import tensorflow as tf

In [2]:
def unet_model(n_classes=1, IMG_HEIGHT=512, IMG_WIDTH=288, IMG_CHANNELS=3):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    #Expansive path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(n_classes, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    return model

In [26]:
# Antiguo train
# Obtén la lista de archivos
#dir_imagenes = r"C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\train\input_images/"
#dir_mascaras = r"C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\train\output_masks/"

#archivos_imagenes = os.listdir(dir_imagenes)
#archivos_mascaras = os.listdir(dir_mascaras)

# Lee imágenes y máscaras y almacénalas en listas
#imagenes = []
#mascaras = []
#for img_file, mask_file in zip(archivos_imagenes, archivos_mascaras):
    
#    img_path = os.path.join(dir_imagenes, img_file)
#    mask_path = os.path.join(dir_mascaras, mask_file)
    
#    img = cv2.imread(img_path)
#    img = cv2.resize(img, (512, 288))
    
#    mask = cv2.imread(mask_path, cv2.IMREAD_ANYDEPTH)
#    mask = cv2.resize(mask, (512, 288))
    
#    imagenes.append(img)
#    mascaras.append(mask[:,:])

# Convierte listas a matrices numpy
#imagenes = np.array(imagenes)
#mascaras = np.array(mascaras)

#mascaras = np.expand_dims(mascaras, axis = -1)

# Divide los datos en entrenamiento y prueba
#X_train, X_test, y_train, y_test = train_test_split(imagenes, mascaras, test_size=0.2, random_state=42)

# Normaliza las imágenes dividiendo por 255
#X_train = X_train.astype('float32')# / 255.
#X_test = X_test.astype('float32')# / 255.

#y_train = y_train.astype('float32') / 1.
#y_test = y_test.astype('float32') / 1.

#modelo = unet_model(n_classes=1, IMG_HEIGHT=X_train.shape[1], IMG_WIDTH=X_train.shape[2], IMG_CHANNELS=X_train.shape[3])

# Compila el modelo
#modelo.compile(optimizer=Adam(), loss=binary_crossentropy, metrics=[MeanIoU(num_classes=2)])

# Define el callback para guardar el modelo con la mejor precisión en el conjunto de prueba
#checkpoint = ModelCheckpoint('modelo_unet.h5', monitor='val_mean_io_u', verbose=1, save_best_only=True, mode='max')

# Entrena el modelo

#historia = modelo.fit(X_train, y_train, batch_size=4, epochs=10, validation_data=(X_test, y_test), callbacks=[checkpoint])
# Guarda el modelo entrenado
#modelo.save(r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\train/unet_Lescala_Unnormalized.h5')

Epoch 1/10
70/70 [==============================] - ETA: 0s - loss: 2.5412 - mean_io_u: 0.4126
Epoch 1: val_mean_io_u improved from -inf to 0.39615, saving model to modelo_unet.h5
70/70 [==============================] - 32s 387ms/step - loss: 2.5412 - mean_io_u: 0.4126 - val_loss: 0.6143 - val_mean_io_u: 0.3961
Epoch 2/10
70/70 [==============================] - ETA: 0s - loss: 0.5242 - mean_io_u: 0.4071
Epoch 2: val_mean_io_u did not improve from 0.39615
70/70 [==============================] - 51s 736ms/step - loss: 0.5242 - mean_io_u: 0.4071 - val_loss: 0.5276 - val_mean_io_u: 0.3961
Epoch 3/10
70/70 [==============================] - ETA: 0s - loss: 0.4766 - mean_io_u: 0.4071
Epoch 3: val_mean_io_u did not improve from 0.39615
70/70 [==============================] - 36s 504ms/step - loss: 0.4766 - mean_io_u: 0.4071 - val_loss: 0.5370 - val_mean_io_u: 0.3961
Epoch 4/10
70/70 [==============================] - ETA: 0s - loss: 0.5335 - mean_io_u: 0.4071
Epoch 4: val_mean_io_u did no

In [27]:
modelo = unet_model(n_classes=1, IMG_HEIGHT=288, IMG_WIDTH=512, IMG_CHANNELS=3)
modelo.load_weights(r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\train/unet_Lescala_Unnormalized.h5')

In [3]:
mascara_agua_path = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\mascara_agua_segmentacion/mascara_agua_lescala_2.jpg'
mascara_javi = r'C:\Users\aquacorp\Desktop\mask_javi.jpg'

In [32]:
modelo = load_model(r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\imagenes_post_mov/unet_ampliado_28sep_16oct.h5')

path_img_ = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\imagenes_post_mov\originales/20241003050943.tif'
mascara_agua_path = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\mascara_agua_segmentacion/mask_lescala_mov_camara.jpg'
#mask_GT_path = r'C:\Users\aquacorp\Desktop\Imagenes\Cidacos_HOMO\Segmentacion/Validacion_set/GT_mask/m_20240322172839.jpg'

img_ = cv2.imread(path_img_)
img_ = cv2.resize(img_, (128, 128))
img_ = np.expand_dims(img_, axis=0)

#mask_GT_ = cv2.imread(mask_GT_path, cv2.IMREAD_ANYDEPTH)
#mask_GT_ = cv2.resize(mask_GT_, (512, 288))

mascara_agua = cv2.imread(mascara_agua_path)
mascara_agua = cv2.resize(mascara_agua, (128, 128))

_, mascara_agua_bin = cv2.threshold(mascara_agua, 128, 1, cv2.THRESH_BINARY)
mascara_agua_bin = np.uint8(mascara_agua_bin)

#_, mask_GT = cv2.threshold(mask_GT_, 128, 1, cv2.THRESH_BINARY)
#mask_GT = np.uint8(mask_GT)

pred_bruto = modelo.predict(img_/255, verbose=False)
pred_bruto = np.squeeze(pred_bruto, axis = 0)
img_ = np.squeeze(img_, axis = 0)
_, pred_bruto = cv2.threshold(pred_bruto, 0.7, 1, cv2.THRESH_BINARY)
pred_bruto = np.uint8(pred_bruto)
pred_bruto_inv = 1 - pred_bruto

mascara_predicha = pred_bruto*mascara_agua_bin[:,:,0]
transparencia = 0.2

mask_pred_rgb = np.zeros_like(img_)
mask_pred_rgb[:,:,2] = mascara_predicha*255

composicion_pred = cv2.addWeighted(mask_pred_rgb, transparencia, img_, 1 - transparencia, 0)
cv2.imshow('Prediccion', composicion_pred)
cv2.imshow('Mascara', mask_pred_rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [31]:
#modelo = unet_model(n_classes=1, IMG_HEIGHT=288, IMG_WIDTH=512, IMG_CHANNELS=3)

modelo = load_model(r'C:\Users\aquacorp\Desktop\espacios_latentes\Lescala\modelo_segmentacion_load_tf/unet_Lescala.h5')
path_imges = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\espumas_azules_22-23sep_2024\originales'
path_results = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\espumas_azules_22-23sep_2024\predicciones_modelo_nuevo'
mascara_agua_path = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\mascara_agua_segmentacion/mascara_agua_lescala_2.jpg'

transparencia = 0.2

mascara_agua = cv2.imread(mascara_agua_path)
#mascara_agua = cv2.resize(mascara_agua, (512, 288))
mascara_agua = cv2.resize(mascara_agua, (288, 512))

_, mascara_agua_bin = cv2.threshold(mascara_agua, 128, 1, cv2.THRESH_BINARY)
mascara_agua_bin = np.uint8(mascara_agua_bin)

total = len(os.listdir(path_imges))
for img in tqdm(os.listdir(path_imges), desc="Generando máscaras", total = total):
    im_pat = os.path.join(path_imges, img)
    img_ = load_img(im_pat, target_size=(512, 288))
    #img_ = cv2.resize(img_, (512, 288))
    #img_ = cv2.resize(img_, (288, 512))
    img_ = np.expand_dims(img_, axis=0)
    
    pred_bruto = modelo.predict(img_/255, verbose=False)
    #pred_bruto = modelo.predict(img_, verbose=False)[0]
    pred_bruto = np.squeeze(pred_bruto, axis = 0)
    #img_ = np.squeeze(img_, axis = 0)
    _, pred_bruto = cv2.threshold(pred_bruto, 0.3, 1, cv2.THRESH_BINARY)
    #pred_bruto = (pred_bruto > 0.5).astype(np.uint8)
    pred_bruto = np.uint8(pred_bruto)
    pred_bruto_inv = 1 - pred_bruto
    #print(mascara_predicha.shape)
    print('pred_bruto')
    print(pred_bruto.shape)
    print('mascara_agua_bin')
    print(mascara_agua_bin.shape)
    mascara_predicha = pred_bruto*mascara_agua_bin[:,:,0]
    print('mascara_predicha')
    print(mascara_predicha.shape)
    mask_pred_rgb = np.zeros(shape=(512, 288, 3), dtype='uint8')
    
    mask_pred_rgb[:,:,2] = mascara_predicha*255
    
    #print()
    img_ = np.squeeze(img_, axis=0)
    composicion_pred = cv2.addWeighted(mask_pred_rgb, transparencia, img_, 1 - transparencia, 0)
    
    # Calculo de % de espuma
    mask_agua_inv = 1 - mascara_agua_bin

    neg_0_y_1 = mascara_predicha - mask_agua_inv[:,:,0]
    
    pixel_agua = np.sum(neg_0_y_1 == 0)
    pixel_espuma = np.sum(neg_0_y_1 == 1)
    
    porcentaje = 100*pixel_espuma/(pixel_agua + pixel_espuma)
    # Fin calculo de % espuma
    
    dir_save_mask = path_results + f'/{int(porcentaje)}%_' + img
    
    cv2.imwrite(dir_save_mask, composicion_pred)
    

Generando máscaras:   1%|          | 2/208 [00:00<01:17,  2.64it/s]

pred_bruto
(512, 288)
mascara_agua_bin
(512, 288, 3)
mascara_predicha
(512, 288)
pred_bruto
(512, 288)
mascara_agua_bin
(512, 288, 3)
mascara_predicha
(512, 288)


Generando máscaras:   2%|▏         | 4/208 [00:00<00:53,  3.80it/s]

pred_bruto
(512, 288)
mascara_agua_bin
(512, 288, 3)
mascara_predicha
(512, 288)
pred_bruto
(512, 288)
mascara_agua_bin
(512, 288, 3)
mascara_predicha
(512, 288)


Generando máscaras:   3%|▎         | 6/208 [00:01<00:42,  4.72it/s]

...

pred_bruto
(512, 288)
mascara_agua_bin
(512, 288, 3)
mascara_predicha
(512, 288)
pred_bruto
(512, 288)
mascara_agua_bin
(512, 288, 3)
mascara_predicha
(512, 288)


Generando máscaras:   6%|▋         | 13/208 [00:02<00:32,  5.93it/s]

pred_bruto
(512, 288)
mascara_agua_bin
(512, 288, 3)
mascara_predicha
(512, 288)
pred_bruto
(512, 288)
mascara_agua_bin
(512, 288, 3)
mascara_predicha
(512, 288)


Generando máscaras:   7%|▋         | 14/208 [00:02<00:38,  5.09it/s]


KeyboardInterrupt: 

Aquí vamos a calcular las métricas. Por un lado se va a calcular la IoU para ver lo bien que funciona el modelo.
Por otro lado se va a dar el resultado 'final' que se proporciona como % de grasas sobre el agua.

Aquí vamos a calcular el % de ocupación de grasas. Se hace de la siguiente manera
Se lee la máscara de agua para acotar la zona. La máscara tiene a 0 los pixeles donde no hay agua y 1 los pixeles
donde sí que hay agua. Se sumarán los pixeles donde sí hay agua para tener el denominador del % de la ocupación de grasas.
Por otro lado, se suman los pixeles en blanco de la máscara predicha, pues en este caso son los píxeles blancos es donde hay grasa.
El resultado de ocupación será el cociente entre las magnitudes descritas multiplicado por 100. 
Este resultado se acompña de  la imagen de la grasa cubierta por la máscara de grasa en versión traslucida.


In [19]:
pixeles_agua_total = np.count_nonzero(mascara_agua_bin == 1)/3
pixeles_grasa_pred = np.count_nonzero(mascara_predicha == 1)
pixeles_grasa_GT = np.count_nonzero(mask_GT == 1)

porcentaje_grasa_pred = 100 * (pixeles_grasa_pred / pixeles_agua_total)
p_grasa_pred = str(int(porcentaje_grasa_pred)) + '%'
print('Porcentaje predicho: ' + p_grasa_pred)

porcentaje_grasa_GT = 100 * (pixeles_grasa_GT / pixeles_agua_total)
p_grasa_GT = str(int(porcentaje_grasa_GT)) + '%'
print('Porcentaje GT: ' + p_grasa_GT)

Porcentaje predicho: 37%
Porcentaje GT: 79%


In [21]:
transparencia = 0.2

mask_pred_rgb = cv2.cvtColor(mascara_predicha*255, cv2.COLOR_GRAY2RGB)
composicion_pred = cv2.addWeighted(mask_pred_rgb, transparencia, img_, 1 - transparencia, 0)

mask_GT_rgb = cv2.cvtColor(mask_GT*255, cv2.COLOR_GRAY2RGB)
composicion_GT = cv2.addWeighted(mask_GT_rgb, transparencia, img_, 1 - transparencia, 0)

cv2.imshow(p_grasa_pred + ' Prediccion', composicion_pred)
cv2.imshow(p_grasa_GT + ' GT', composicion_GT)
cv2.waitKey(0)
cv2.destroyAllWindows()

A partir de aquí se van a hacer calculos de IoU. La filosofía de este cálculo será sumar la máscara Ground Truth y la máscara
predicha. Los puntos donde coincidan 1 dará como resultado 2, los puntos donde no coincidan se mantendrán como 1 y se quedaran
en 0 las zonas donde coincidan en que no hay grasas. La intersección serían todos los pixeles donde el valor es 2, la unión será
todos los pixeles que tengan valor 1 o 2. La métrica IoU será el cociente interseccion/union. Se puede expresar en % o entre 0 y 1.  

In [161]:
suma_GT_pred = mask_GT + mascara_predicha
I = np.count_nonzero(suma_GT_pred == 2)
U = np.count_nonzero(suma_GT_pred == 2) + np.count_nonzero(suma_GT_pred == 1)
IoU = I/U
print(IoU*100)

31.659517955528436
